# Data Preparation & Cleaning

The raw job-posting dataset is prepared into a clean file that is used in all downstream analysis notebooks.

## Import Libraries and Data

In [14]:
import numpy as np
import pandas as pd
import ast

df = pd.read_csv('E:/Projects/Python_Data_Project/job_postings_flat.csv')

## Raw Data Snapshot

In [15]:
df.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Data Analyst,Data Analyst,"New York, NY",via CareerBuilder,Full-time,False,"New York, United States",2023-01-01 00:00:04,False,False,United States,NaN,NaN,NaN,Metasys Technologies,"['sql', 'snowflake', 'visio', 'jira', 'conflue...","{'analyst_tools': ['visio'], 'async': ['jira',..."
1,Data Analyst,Data Analyst,"Washington, DC",via CareerBuilder,Full-time,False,"New York, United States",2023-01-01 00:00:22,False,True,United States,NaN,NaN,NaN,Guidehouse,"['sql', 'python', 'r', 'azure', 'snowflake', '...","{'analyst_tools': ['tableau', 'excel'], 'cloud..."
2,Data Analyst,Data Analyst,"Fairfax, VA",via CareerBuilder,Full-time,False,"New York, United States",2023-01-01 00:00:24,False,False,United States,NaN,NaN,NaN,Protask,"['sql', 'jira']","{'async': ['jira'], 'programming': ['sql']}"
3,Senior Data Analyst,Senior Data Analyst / Platform Experience,"Worcester, MA",via LinkedIn,Full-time,False,"New York, United States",2023-01-01 00:00:27,False,True,United States,NaN,NaN,NaN,Atria Wealth Solutions,"['sql', 'atlassian', 'jira']","{'async': ['jira'], 'other': ['atlassian'], 'p..."
4,Data Analyst,Jr. Data Analyst,"Torrance, CA",via Recruit.net,Full-time,False,"California, United States",2023-01-01 00:00:38,False,False,United States,NaN,NaN,NaN,Aquent,['excel'],{'analyst_tools': ['excel']}


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780669 entries, 0 to 1780668
Data columns (total 17 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   job_title_short        object 
 1   job_title              object 
 2   job_location           object 
 3   job_via                object 
 4   job_schedule_type      object 
 5   job_work_from_home     bool   
 6   search_location        object 
 7   job_posted_date        object 
 8   job_no_degree_mention  bool   
 9   job_health_insurance   bool   
 10  job_country            object 
 11  salary_rate            object 
 12  salary_year_avg        float64
 13  salary_hour_avg        float64
 14  company_name           object 
 15  job_skills             object 
 16  job_type_skills        object 
dtypes: bool(3), float64(2), object(12)
memory usage: 195.3+ MB


## Create Working Copy

Create a separate working dataframe so all changes are applied in one place.

In [17]:
df_raw = df.copy()
df_work = df_raw.copy()

## Basic Text Standardization

Standardize key text fields to reduce inconsistencies caused by formatting differences.

In [18]:
def clean_text_series(s: pd.Series) -> pd.Series:
    return (
        s.astype("string")
         .str.replace(r"\s+", " ", regex=True)
         .str.strip()
    )

text_cols = [
    "job_title",
    "company_name",
    "search_location",
    "job_location",
    "job_schedule_type",
    "job_country",
    "job_title_short",
    "job_via",
]

df_before = df_work.copy()

for col in text_cols:
    df_work[col] = clean_text_series(df_work[col])

print("\nTEXT STANDARDIZATION – VALUE CHANGES")
for col in text_cols:
    changed = (df_before[col] != df_work[col]) & ~(df_before[col].isna() & df_work[col].isna())
    print(f"{col}: {changed.sum()} values changed")


TEXT STANDARDIZATION – VALUE CHANGES
job_title: 8917 values changed
company_name: 1700 values changed
search_location: 0 values changed
job_location: 4354 values changed
job_schedule_type: 0 values changed
job_country: 0 values changed
job_title_short: 0 values changed
job_via: 0 values changed


## Dates and Weekly Bucket

Convert posting dates to datetime and create a weekly bucket used for time grouping and deduplication.

In [19]:
df_work["job_posted_date"] = pd.to_datetime(df_work["job_posted_date"], errors="coerce")
df_work["posted_week"] = df_work["job_posted_date"].dt.to_period("W").astype(str)

## Job Posting Platform

Clean and normalize job source labels to make grouping by source more consistent.

In [20]:
df_work["job_via"] = (
    df_work["job_via"]
        .str.replace(r"^via\s+", "", regex=True, case=False)
        .str.strip()
)

## Company name standardization

Standardize company names to reduce duplicated labels caused by minor variations.

In [21]:
df_work["company_name"] = df_work["company_name"].fillna("Unknown").astype(str).str.strip()
df_work["company_name"] = df_work["company_name"].str.replace(r"\s+", " ", regex=True)

s = df_work["company_name"].str.lower()

df_work["company_name"] = np.where(s.str.contains("bebee"), "BeBee", df_work["company_name"])
df_work["company_name"] = np.where(s.str.contains("jobs via dice") | s.eq("dice"), "Dice", df_work["company_name"])
df_work["company_name"] = np.where(s.str.contains("confidential"), "Confidential", df_work["company_name"])

## Week-Based Deduplication

Remove repost-style duplicates within the same week using a composite deduplication key.

In [22]:
def report_step(step_name: str, before_rows: int, after_rows: int) -> None:
    delta = after_rows - before_rows
    removed_pct = (1 - after_rows / before_rows) * 100 if before_rows else 0
    print(f"\n=== {step_name} ===")
    print(f"Rows before: {before_rows:,}")
    print(f"Rows after:  {after_rows:,}")
    print(f"Δ rows:      {delta:,}")
    print(f"Removed (%): {removed_pct:.2f}%")

dedup_cols = [
    "job_title",
    "company_name",
    "search_location",
    "job_schedule_type",
    "job_work_from_home",
    "posted_week",
]

df_cleaned = (
    df_work
        .sort_values("job_posted_date")
        .drop_duplicates(subset=dedup_cols, keep="last")
        .copy()
)

report_step("WEEK-BASED DEDUPLICATION", len(df_work), len(df_cleaned))


=== WEEK-BASED DEDUPLICATION ===
Rows before: 1,780,669
Rows after:  1,550,333
Δ rows:      -230,336
Removed (%): 12.94%


## Deduplicate Skills Within a Posting

- Remove repeated skills within each posting so later explode()-based analyses do not overcount skill mentions.

- Parse job_skills from its raw CSV string format (stringified Python list) into an actual Python list using ast.literal_eval, matching the parsing logic used in downstream notebooks.

- Deduplicate each skills list while preserving the original order of first occurrence (keeps the “first-seen” skill sequence intact).

- Report the duplicate-skill posting rate before and after deduplication, computed only for rows where job_skills is a non-empty list.

In [24]:
df_cleaned["job_skills"] = df_cleaned["job_skills"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

def dup_rate(series: pd.Series) -> float:
    mask = series.apply(lambda x: isinstance(x, list) and len(x) > 0)
    return series[mask].apply(lambda lst: len(lst) != len(set(lst))).mean()

before = dup_rate(df_cleaned["job_skills"])

df_cleaned["job_skills"] = df_cleaned["job_skills"].apply(
    lambda lst: list(dict.fromkeys(lst)) if isinstance(lst, list) else lst
)

after = dup_rate(df_cleaned["job_skills"])

print(f"Duplicate-skill postings BEFORE dedupe: {before:.2%}")
print(f"Duplicate-skill postings AFTER  dedupe: {after:.2%}")

Duplicate-skill postings BEFORE dedupe: 8.49%
Duplicate-skill postings AFTER  dedupe: 0.00%


## Drop Helper Columns

Drop intermediate helper columns that are no longer needed after preprocessing.

In [ ]:
df_cleaned = df_cleaned.drop(columns=["posted_week"])

## Export Clean Dataset

- Export the cleaned dataset to a single file used across the project.

- **Key normalizations**: job_posted_date → datetime, job_via cleaned (removed “via …”), company_name standardized.

- **Deduplication**: week-based dedupe on job_title + company_name + search_location + schedule + WFH + posted_week.

- **Skills**: duplicates inside a single posting removed; stored in CSV as stringified lists (parsed later with ast.literal_eval)

In [25]:
print("\nFINAL DATASET SUMMARY")
print("Raw rows:", len(df_raw))
print("Final rows:", len(df_cleaned))
print("Total rows removed:", len(df_raw) - len(df_cleaned))

df_cleaned.to_csv("job_postings_flat_cleaned.csv", index=False)
print("Saved: job_postings_flat_cleaned.csv")


FINAL DATASET SUMMARY
Raw rows: 1780669
Final rows: 1550333
Total rows removed: 230336
Saved: job_postings_flat_cleaned.csv
